In [1]:
from lstm_creator import createLstmModelFromDatasets, scaleData, preprocessData
from predict_next_value import predict_next, load_x_from_file

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# use the last 90 minutes to predict next value
lookback_length = 75

# data is stores from every 15 seconds, but we only want every 4th item
sample_every_x = 3

paths = ["../../../data/dump.csv"]
model = createLstmModelFromDatasets(paths, lookback_length, sample_every_x, epochs=360)

# Uncomment this line to save the model on completion
# model.save('../../../models/dump_model_3_ll20_xth3')

==========================] - 8s 38ms/step - loss: 5.4560e-04
Epoch 137/360
213/213 [==============================] - 8s 40ms/step - loss: 5.6454e-04
Epoch 138/360
213/213 [==============================] - 8s 39ms/step - loss: 5.4780e-04
Epoch 139/360
213/213 [==============================] - 9s 42ms/step - loss: 5.8887e-04
Epoch 140/360
213/213 [==============================] - 9s 41ms/step - loss: 5.4007e-04
Epoch 141/360
213/213 [==============================] - 10s 48ms/step - loss: 5.6052e-04
Epoch 142/360
213/213 [==============================] - 9s 42ms/step - loss: 5.3532e-04
Epoch 143/360
213/213 [==============================] - 9s 42ms/step - loss: 5.7874e-04
Epoch 144/360
213/213 [==============================] - 9s 43ms/step - loss: 5.4912e-04
Epoch 145/360
213/213 [==============================] - 9s 44ms/step - loss: 5.5506e-04
Epoch 146/360
213/213 [==============================] - 9s 42ms/step - loss: 5.7735e-04
Epoch 147/360
213/213 [========================

# Load model

In [3]:
from tensorflow.keras import models
from lstm_creator import createLstmModelFromDatasets, scaleData, preprocessData
import numpy as np

# Uncomment this line to load the model from a file for testing
# model = models.load_model('../../../models/dump_model_1')

# Testing

In [13]:
def evaluate_model(real_data, predicted_data, slope_to_buy_at, inherent_loss=2):
    real_data = real_data.reshape(len(real_data))
    predicted_data = predicted_data.reshape(len(predicted_data))

    real_diff = np.diff(real_data)
    predicted_diff = np.diff(predicted_data)

    profit = 0
    profits = 0
    losses = 0
    bought = False
    errors = []
    for i in range(len(real_data)-1):
            #  Error calculation
            # if real_diff[i] != 0:
            errors.append( 1 if np.sign(real_diff[i]) == np.sign(predicted_diff[i]) else 0)
                # errors.append( abs(real_diff[i] - predicted_diff[i]) / abs(real_diff[i]) )

            if predicted_diff[i] > slope_to_buy_at:
                if bought == False:
                    bought = real_data[i]
                else:
                    # Already bought, continue
                    continue
            if bought != False and predicted_diff[i] < slope_to_buy_at:
                # Now we sell
                profit += (real_data[i+1] - real_data[i]) - inherent_loss
                if ((real_data[i+1] - real_data[i]) - inherent_loss) > 0:
                    profits += 1
                else:
                    losses += 1
                bought = False

    return (errors, profit, profits, losses)

def eval_model_on_dataset(actual, prediction, slope_to_buy_at, subsampling, inherent_loss):
    # Determine the number of cases in which we predicted a correct increase
    errors, profit, profits, losses = evaluate_model(actual, prediction, slope_to_buy_at, inherent_loss)

    # print(f"Found {correct_slopes} out of {len(actual)-1}")
    percent_error = (sum(errors)/len(errors))*100
    # print(f"{precent_success}%")
    # print("Profit:", profit)
    return percent_error, profit, profits, losses

In [14]:
def testModel(model, length, slope_to_buy_at, path_to_testing_dataset, quiet=False):

    datasets, scaler = scaleData([path_to_testing_dataset], 3)

    hist, actual = preprocessData(datasets[0], length)

    pred = model.predict(hist)

    pred_transformed = scaler.inverse_transform(pred)
    actual_transformed = scaler.inverse_transform(actual)

    if not quiet:
        plt.figure(figsize=(12,8))
        plt.plot(actual_transformed, color='blue', label='Real')
        plt.plot(pred_transformed, color='red', label='Prediction')
        plt.title('ETH Price Prediction')
        plt.legend()
        plt.show()

    return eval_model_on_dataset(actual=actual_transformed, prediction=pred_transformed, slope_to_buy_at=slope_to_buy_at, subsampling=1, inherent_loss=1.5)

In [15]:
overall = (0, 0, 0, 0)
lookback_length = model.input_shape[1]
slope_to_buy_at = 2


result1 = testModel(model, lookback_length, slope_to_buy_at, "../../../data/test_data/test6.csv", quiet=True)
accuracy, profit, profits, losses = result1
print(f"Error: {accuracy:.2f}% Profit: ${profit:.2f} (Profits/Losses: {profits}/{losses})")

result2 = testModel(model, lookback_length, slope_to_buy_at, "../../../data/test_data/test8.csv", quiet=True)
accuracy, profit, profits, losses = result2
print(f"Error: {accuracy:.2f}% Profit: ${profit:.2f} (Profits/Losses: {profits}/{losses})")

result3 = testModel(model, lookback_length, slope_to_buy_at, "../../../data/test_data/test10.csv", quiet=True)
accuracy, profit, profits, losses = result3
print(f"Error: {accuracy:.2f}% Profit: ${profit:.2f} (Profits/Losses: {profits}/{losses})")

overall = [sum(x) for x in zip(result1,overall)]
overall = [sum(x) for x in zip(result2,overall)]
overall = [sum(x) for x in zip(result3,overall)]
print("-----------------------------------")
print(f"Error: {overall[0]/3:.2f}% Profit: ${overall[1]:.2f} (Profits/Losses: {overall[2]}/{overall[3]})")



Error: 48.66% Profit: $-8.18 (Profits/Losses: 1/5)
Error: 48.55% Profit: $-14.35 (Profits/Losses: 4/6)
Error: 52.61% Profit: $-13.64 (Profits/Losses: 6/9)
-----------------------------------
Error: 49.94% Profit: $-36.17 (Profits/Losses: 11/20)
